In [508]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import copy
from math import sqrt,ceil

from rope.base.oi.soa import analyze_module
from scipy.stats import linregress
import libraries as lib

In [509]:
directory = "../../../OneDrive - HvA/Logfiles/H2A/2022/20220628-20220701_race_nogaro/0023 - 20220701T141014 1h8m8s 13.336km Nogaro/"
file = "1.csv"

In [510]:
data_file = lib.DataInladen(f'{directory}{file}')

In [511]:
data_file

,Datalogger port,"Dataloggertijd, in s",Format header (>08|03),NMEA type($GPRMC),Tijd,Status,Latitude,Noord of Zuid (N/S),Longitude,Oost of West (E/W),...,Mode indicator (A),Data in balancers,DegreesTrue,T,DegreesMagnetic,M,Snelheid tenopzichte van het water in Knots,N,Snelheid tenopzichte van het water in Kilometers,K
0,1,70.8626,08|03,$GPRMC,120813.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,71.8578,08|03,$GPRMC,120814.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,72.8577,08|03,$GPRMC,120815.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,73.8609,08|03,$GPRMC,120816.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,74.8585,08|03,$GPRMC,120817.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,1,4276.0450,08|03,$GPRMC,131818.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4201,1,4277.0633,08|03,$GPRMC,131819.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4202,1,4278.0288,08|03,$GPRMC,131820.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4203,1,4279.0253,08|03,$GPRMC,131821.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [512]:
# Filtering the data so it's only the laps we want to analyze
# First filter so that the data without GPS data is removed
data_file = data_file[data_file[xas] != 0]
data_file = data_file[data_file[yas] != 0]

# Now filter out the beginning and end of the data, by removing the first and last 10% of the data
from math import ceil
data_file
data_file = data_file.iloc[ceil(len(data_file)*0.215):ceil(len(data_file)*0.267)]

# Display the filtered data
data_file

,Datalogger port,"Dataloggertijd, in s",Format header (>08|03),NMEA type($GPRMC),Tijd,Status,Latitude,Noord of Zuid (N/S),Longitude,Oost of West (E/W),...,Mode indicator (A),Data in balancers,DegreesTrue,T,DegreesMagnetic,M,Snelheid tenopzichte van het water in Knots,N,Snelheid tenopzichte van het water in Kilometers,K
1000,1,1074.0251,08|03,$GPRMC,122456.0,A,4346.2791,N,2.4708,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,1,1075.0263,08|03,$GPRMC,122457.0,A,4346.2825,N,2.4744,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,1,1076.0461,08|03,$GPRMC,122458.0,A,4346.2856,N,2.4783,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,1,1077.0251,08|03,$GPRMC,122459.0,A,4346.2890,N,2.4821,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,1,1078.0262,08|03,$GPRMC,122500.0,A,4346.2920,N,2.4858,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,1,1282.0243,08|03,$GPRMC,122824.0,A,4346.2801,N,2.4701,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1209,1,1283.0397,08|03,$GPRMC,122825.0,A,4346.2833,N,2.4742,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1210,1,1284.0313,08|03,$GPRMC,122826.0,A,4346.2865,N,2.4780,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1211,1,1285.0255,08|03,$GPRMC,122827.0,A,4346.2896,N,2.4820,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [513]:
'Plotting the race'
plt.figure(figsize=(6,6))
plt.scatter(data_file['Latitude'], data_file['Longitude'], c=data_file['Snelheid over de grond in km/h'], cmap='viridis')
plt.title('GPS data vs speed')
plt.xlabel('Latitude')
plt.ylabel('Longitude')

# Adding a colorbar with a label for the speed in km/h
cbar = plt.colorbar()
cbar.set_label('Speed (km/h)')
plt.grid()
plt.show()

In [514]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Extract data for plotting
latitudes = data_file['Latitude'].values
longitudes = data_file['Longitude'].values
speeds = data_file['Snelheid over de grond in km/h'].values

# Plotting the race with a 3D scatter plot
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with height representing speed
scatter = ax.scatter(latitudes, longitudes, speeds, c=speeds, cmap='viridis', s=10)

# Adding labels and title
ax.set_title('GPS data vs speed')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
ax.set_zlabel('Speed (km/h)')

# Adding a colorbar with a label for the speed in km/h
cbar = plt.colorbar(scatter, ax=ax, pad=0.1)
cbar.set_label('Speed (km/h)')

plt.show()